In [1]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from imutils import face_utils
font = cv2.FONT_HERSHEY_SIMPLEX

In [2]:
import os

def get_files(path):
    return os.listdir(path)

cascPath = "/Users/abdulrehman/opt/anaconda3/envs/Face-Detection/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml"

def return_bbx(image):
    faceCascade = cv2.CascadeClassifier(cascPath)
    faces = faceCascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, flags=cv2.CASCADE_SCALE_IMAGE)
    return faces

In [3]:
Dataset_path = '/Users/abdulrehman/Desktop/SML Project/FacesInTheWild/'

Celebs = pd.read_csv(Dataset_path+'lfw_allnames.csv')
Celebs = Celebs[Celebs['images']>10]
Celebs

,name,images
20,Abdullah_Gul,19
52,Adrien_Brody,12
127,Alejandro_Toledo,39
210,Alvaro_Uribe,35
223,Amelie_Mauresmo,21
...,...,...
5541,Vicente_Fox,32
5569,Vladimir_Putin,49
5605,Wen_Jiabao,13
5659,Winona_Ryder,24


In [4]:
cat_list = []
X = []
Y = []
y_label = 0

for _, [name,__] in Celebs.iterrows():
    celeb_path = Dataset_path+'lfw-deepfunneled/'+name+'/'
    
    images_paths = get_files(celeb_path)
    temp = []
    for image_path in images_paths:
        image = cv2.imread(celeb_path+image_path,1)
        faces = return_bbx(image)
        if len(faces) == 1:
            if len(temp)>=10:
                break
            temp.append(len(X))
            (x,y,w,h) = faces[0]
            cropped = image[x:x+w, y:y+h]
            dim = (100, 100)
            resized = cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
            image = np.array(resized).astype("float32")
            X.append(image)
            Y.append(y_label)
    y_label+=1
    cat_list.append(temp)
        



In [9]:
X_data = np.asarray(X)/255
Y_data = np.array(Y)
cat_list = np.asarray(cat_list)

print(X_data.shape, Y_data.shape)

(1423, 100, 100, 3) (1423,)


In [10]:
from collections import Counter

Counter(Y_data)

Counter({0: 10,
         1: 10,
         2: 10,
         3: 10,
         4: 10,
         5: 10,
         6: 10,
         7: 10,
         8: 10,
         9: 10,
         10: 10,
         11: 10,
         12: 10,
         13: 10,
         14: 10,
         15: 10,
         16: 10,
         17: 10,
         18: 10,
         19: 10,
         20: 10,
         21: 10,
         22: 10,
         23: 10,
         24: 10,
         25: 10,
         26: 10,
         27: 10,
         28: 10,
         29: 10,
         30: 10,
         31: 10,
         32: 10,
         33: 10,
         34: 10,
         35: 10,
         36: 10,
         37: 9,
         38: 10,
         39: 10,
         40: 10,
         41: 10,
         42: 10,
         43: 10,
         44: 10,
         45: 10,
         46: 10,
         47: 10,
         48: 10,
         49: 10,
         50: 10,
         51: 10,
         52: 10,
         53: 10,
         54: 10,
         55: 10,
         56: 10,
         57: 10,
         58: 10,
        

In [11]:

a = Y_data
n_classes = len(set(a))
n_classes

143

In [12]:
train_split = 0.9

train_size = int(n_classes*train_split)
test_size = n_classes-train_size

train_files = train_size * 10

X_train = X_data[:train_files]
y_train = Y_data[:train_files]
cat_train = cat_list[:train_size]

#Validation Split
X_test = X_data[train_files:]
y_test = Y_data[train_files:]
cat_test = cat_list[train_size:]

print('X&Y shape of training data :',X_train.shape, 'and', y_train.shape, cat_train.shape)
print('X&Y shape of testing data :' , X_test.shape, 'and', y_test.shape, cat_test.shape)

X&Y shape of training data : (1280, 100, 100, 3) and (1280,) (128,)
X&Y shape of testing data : (143, 100, 100, 3) and (143,) (15,)


In [51]:
def get_batch(batch_size=64):
    
    temp_x = X_train
    temp_cat_list = cat_train
    start=0
    end=train_size
    batch_x=[]
        
    batch_y = np.zeros(batch_size)
    batch_y[int(batch_size/2):] = 1
    np.random.shuffle(batch_y)
    
    class_list = np.random.randint(start, end, batch_size) 
    batch_x.append(np.zeros((batch_size, 100, 100, 3)))
    batch_x.append(np.zeros((batch_size, 100, 100, 3)))

    for i in range(0, batch_size):
        batch_x[0][i] = temp_x[np.random.choice(temp_cat_list[class_list[i]])]  
        #If train_y has 0 pick from the same class, else pick from any other class
        if batch_y[i]==0:
            batch_x[1][i] = temp_x[np.random.choice(temp_cat_list[class_list[i]])]

        else:
            temp_list = np.append(temp_cat_list[:class_list[i]].flatten(), temp_cat_list[class_list[i]+1:].flatten())
            temp_list = np.random.choice(temp_list)
            batch_x[1][i] = temp_x[np.random.choice(temp_list)]
            
            
    return(batch_x, batch_y)

In [65]:
from keras.applications import VGG16
from keras.layers import Dense, Dropout, Activation, Flatten, Subtract, Input, BatchNormalization
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import plot_model
# from keras.layers.advanced_activations import LeakyReLU

input_shape = (100,100,3)

vgg_left = VGG16(weights = 'imagenet',include_top = False, input_shape = input_shape)

for layer in vgg_left.layers:
    layer.trainable = False
    layer._name = 'left_'+layer.name
    
left = [layer.output for layer in vgg_left.layers][-7]

left = Flatten()(left)
left = Dropout(0.5)(left)
left = Dense(900, kernel_regularizer=l2(1e-2))(left)
left = BatchNormalization()(left)
left = Activation('relu')(left)


vgg_right = VGG16(weights = 'imagenet',include_top = False, input_shape = input_shape)

for layer in vgg_right.layers:
    layer.trainable = False
    layer._name = 'right_'+layer.name

right = [layer.output for layer in vgg_right.layers][-7]

right = Flatten()(right)
right = Dropout(0.5)(right)
right = Dense(900, kernel_regularizer=l2(1e-2))(right)
right = BatchNormalization()(right)
right = Activation('relu')(right)


subtracted = Subtract()([left,right])
out = Dense(1, activation='sigmoid')(subtracted)

model = Model(inputs = [vgg_left.input,vgg_right.input], outputs = out)

model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=['accuracy'])

model.summary()

Model: "functional_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
left_input_18 (InputLayer)      [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
right_input_19 (InputLayer)     [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
left_block1_conv1 (Conv2D)      (None, 100, 100, 64) 1792        left_input_18[0][0]              
__________________________________________________________________________________________________
right_block1_conv1 (Conv2D)     (None, 100, 100, 64) 1792        right_input_19[0][0]             
______________________________________________________________________________________

In [59]:
def one_shot_learning(model, n_way, n_val):
    
    temp_x = X_test
    temp_cat_list = cat_test
    batch_x=[]
    x_0_choice=[]
    n_correct = 0
   
    class_list = np.random.randint(train_size+1, n_classes-1, n_val)

    for i in class_list:  
        j = np.random.choice(cat_list[i])
        temp=[]
        temp.append(np.zeros((n_way, 100, 100, 3)))
        temp.append(np.zeros((n_way, 100, 100, 3)))
        for k in range(0, n_way):
            temp[0][k] = X_data[j]
            
            if k==0:
                temp[1][k] = X_data[np.random.choice(cat_list[i])]
            else:
                temp_list = np.append(cat_list[:i].flatten(), cat_list[i+1:].flatten())
                temp_list = np.random.choice(temp_list)
                temp[1][k] = X_data[np.random.choice(temp_list)]

        result = model.predict(temp)
        result = result.flatten().tolist()
        result_index = result.index(min(result))
        if result_index == 0:
            n_correct = n_correct + 1
    print(n_correct, "correctly classified among", n_val)
    accuracy = (n_correct*100)/n_val
    return accuracy

In [66]:
epochs = 10000
n_way = 20
n_val = 64
batch_size = 64

loss_list=[]
accuracy_list=[]
for epoch in range(1,epochs):
    batch_x, batch_y = get_batch(batch_size)
    loss = model.train_on_batch(batch_x, batch_y)
    loss_list.append((epoch,loss))
    print('Epoch:', epoch, ', Loss:',loss)
    if epoch%250 == 0:
        print("=============================================")
        accuracy = one_shot_learning(model, n_way, n_val)
        accuracy_list.append((epoch, accuracy))
        print('Accuracy as of', epoch, 'epochs:', accuracy)
        print("=============================================")
        if(accuracy>99):
            print("Achieved more than 90% Accuracy")

Epoch: 1 , Loss: [36.413150787353516, 0.5]
Epoch: 2 , Loss: [36.19757843017578, 0.53125]
Epoch: 3 , Loss: [35.958553314208984, 0.59375]
Epoch: 4 , Loss: [35.629783630371094, 0.46875]
Epoch: 5 , Loss: [35.264427185058594, 0.46875]
Epoch: 6 , Loss: [34.9962158203125, 0.484375]
Epoch: 7 , Loss: [34.607017517089844, 0.484375]
Epoch: 8 , Loss: [34.0762939453125, 0.484375]
Epoch: 9 , Loss: [33.96660614013672, 0.46875]
Epoch: 10 , Loss: [33.522891998291016, 0.453125]
Epoch: 11 , Loss: [33.204803466796875, 0.5]
Epoch: 12 , Loss: [32.95225524902344, 0.453125]
Epoch: 13 , Loss: [32.328521728515625, 0.59375]
Epoch: 14 , Loss: [32.29762268066406, 0.53125]
Epoch: 15 , Loss: [32.1262092590332, 0.46875]
Epoch: 16 , Loss: [31.531391143798828, 0.484375]
Epoch: 17 , Loss: [31.467618942260742, 0.53125]
Epoch: 18 , Loss: [31.27994155883789, 0.515625]
Epoch: 19 , Loss: [31.004077911376953, 0.546875]
Epoch: 20 , Loss: [30.675342559814453, 0.453125]
Epoch: 21 , Loss: [30.415199279785156, 0.578125]
Epoch: 22 

Epoch: 172 , Loss: [13.570953369140625, 0.453125]
Epoch: 173 , Loss: [13.2342529296875, 0.515625]
Epoch: 174 , Loss: [13.668811798095703, 0.34375]
Epoch: 175 , Loss: [13.416696548461914, 0.4375]
Epoch: 176 , Loss: [13.083231925964355, 0.59375]
Epoch: 177 , Loss: [13.134989738464355, 0.546875]
Epoch: 178 , Loss: [13.085309982299805, 0.484375]
Epoch: 179 , Loss: [13.01218032836914, 0.59375]
Epoch: 180 , Loss: [13.3836030960083, 0.421875]
Epoch: 181 , Loss: [12.926779747009277, 0.515625]
Epoch: 182 , Loss: [12.900328636169434, 0.59375]
Epoch: 183 , Loss: [13.002784729003906, 0.5]
Epoch: 184 , Loss: [12.774658203125, 0.546875]
Epoch: 185 , Loss: [12.826277732849121, 0.484375]
Epoch: 186 , Loss: [12.756367683410645, 0.53125]
Epoch: 187 , Loss: [12.50297737121582, 0.578125]
Epoch: 188 , Loss: [12.543216705322266, 0.5625]
Epoch: 189 , Loss: [12.639472961425781, 0.5]
Epoch: 190 , Loss: [12.692861557006836, 0.46875]
Epoch: 191 , Loss: [12.660715103149414, 0.5]
Epoch: 192 , Loss: [12.34932327270

Epoch: 340 , Loss: [7.50798225402832, 0.4375]
Epoch: 341 , Loss: [7.413730621337891, 0.59375]
Epoch: 342 , Loss: [7.435927867889404, 0.484375]
Epoch: 343 , Loss: [7.542851448059082, 0.484375]
Epoch: 344 , Loss: [7.238460540771484, 0.609375]
Epoch: 345 , Loss: [7.355165958404541, 0.484375]
Epoch: 346 , Loss: [7.221383094787598, 0.5]
Epoch: 347 , Loss: [7.164600372314453, 0.59375]
Epoch: 348 , Loss: [7.392096519470215, 0.421875]
Epoch: 349 , Loss: [7.224465370178223, 0.53125]
Epoch: 350 , Loss: [7.164438724517822, 0.546875]
Epoch: 351 , Loss: [7.323452949523926, 0.46875]
Epoch: 352 , Loss: [7.151773929595947, 0.53125]
Epoch: 353 , Loss: [6.99727725982666, 0.625]
Epoch: 354 , Loss: [7.200167179107666, 0.4375]
Epoch: 355 , Loss: [7.2054033279418945, 0.421875]
Epoch: 356 , Loss: [7.109163284301758, 0.5]
Epoch: 357 , Loss: [7.06820011138916, 0.484375]
Epoch: 358 , Loss: [7.200517654418945, 0.453125]
Epoch: 359 , Loss: [6.960214614868164, 0.546875]
Epoch: 360 , Loss: [7.207374572753906, 0.375

Epoch: 509 , Loss: [4.7200822830200195, 0.46875]
Epoch: 510 , Loss: [4.690189838409424, 0.40625]
Epoch: 511 , Loss: [4.42103385925293, 0.609375]
Epoch: 512 , Loss: [4.739438533782959, 0.421875]
Epoch: 513 , Loss: [4.512754440307617, 0.546875]
Epoch: 514 , Loss: [4.641388416290283, 0.421875]
Epoch: 515 , Loss: [4.682775020599365, 0.390625]
Epoch: 516 , Loss: [4.467435836791992, 0.546875]
Epoch: 517 , Loss: [4.563045978546143, 0.40625]
Epoch: 518 , Loss: [4.648697853088379, 0.453125]
Epoch: 519 , Loss: [4.487810134887695, 0.5625]
Epoch: 520 , Loss: [4.411172389984131, 0.546875]
Epoch: 521 , Loss: [4.425206184387207, 0.59375]
Epoch: 522 , Loss: [4.527921199798584, 0.421875]
Epoch: 523 , Loss: [4.524209976196289, 0.484375]
Epoch: 524 , Loss: [4.462250232696533, 0.46875]
Epoch: 525 , Loss: [4.359433650970459, 0.53125]
Epoch: 526 , Loss: [4.5386962890625, 0.40625]
Epoch: 527 , Loss: [4.516570568084717, 0.421875]
Epoch: 528 , Loss: [4.319839000701904, 0.515625]
Epoch: 529 , Loss: [4.374917030

Epoch: 680 , Loss: [3.214555263519287, 0.375]
Epoch: 681 , Loss: [3.218021869659424, 0.40625]
Epoch: 682 , Loss: [3.1216390132904053, 0.484375]
Epoch: 683 , Loss: [3.185838222503662, 0.375]
Epoch: 684 , Loss: [3.1630334854125977, 0.5]
Epoch: 685 , Loss: [2.952378273010254, 0.5625]
Epoch: 686 , Loss: [3.1790051460266113, 0.484375]
Epoch: 687 , Loss: [3.159846782684326, 0.421875]
Epoch: 688 , Loss: [3.0692853927612305, 0.484375]
Epoch: 689 , Loss: [3.002450466156006, 0.578125]
Epoch: 690 , Loss: [2.950991153717041, 0.578125]
Epoch: 691 , Loss: [3.1939291954040527, 0.484375]
Epoch: 692 , Loss: [3.1499037742614746, 0.421875]
Epoch: 693 , Loss: [3.159487247467041, 0.453125]
Epoch: 694 , Loss: [2.9845614433288574, 0.5625]
Epoch: 695 , Loss: [3.049259901046753, 0.5]
Epoch: 696 , Loss: [3.0962376594543457, 0.4375]
Epoch: 697 , Loss: [3.245858669281006, 0.4375]
Epoch: 698 , Loss: [3.1056504249572754, 0.421875]
Epoch: 699 , Loss: [2.950580596923828, 0.609375]
Epoch: 700 , Loss: [2.99866056442260

...

Epoch: 2157 , Loss: [1.0289047956466675, 0.390625]
Epoch: 2158 , Loss: [0.9343730211257935, 0.484375]
Epoch: 2159 , Loss: [0.9534450173377991, 0.515625]
Epoch: 2160 , Loss: [0.897173285484314, 0.484375]
Epoch: 2161 , Loss: [1.0103468894958496, 0.453125]
Epoch: 2162 , Loss: [0.9708115458488464, 0.515625]
Epoch: 2163 , Loss: [0.965408444404602, 0.4375]
Epoch: 2164 , Loss: [1.0375455617904663, 0.4375]
Epoch: 2165 , Loss: [0.9753469228744507, 0.4375]
Epoch: 2166 , Loss: [0.9143958687782288, 0.546875]
Epoch: 2167 , Loss: [1.0065100193023682, 0.46875]
Epoch: 2168 , Loss: [0.9857486486434937, 0.4375]
Epoch: 2169 , Loss: [0.9795373678207397, 0.53125]
Epoch: 2170 , Loss: [0.9857946634292603, 0.453125]
Epoch: 2171 , Loss: [0.9861873388290405, 0.484375]
Epoch: 2172 , Loss: [0.9027059674263, 0.546875]
Epoch: 2173 , Loss: [1.0232596397399902, 0.4375]
Epoch: 2174 , Loss: [1.0110664367675781, 0.4375]
Epoch: 2175 , Loss: [0.917251706123352, 0.59375]
Epoch: 2176 , Loss: [0.9762202501296997, 0.5]
Epoch:

Epoch: 2319 , Loss: [0.9833595156669617, 0.390625]
Epoch: 2320 , Loss: [0.8779541254043579, 0.625]
Epoch: 2321 , Loss: [0.9458868503570557, 0.578125]
Epoch: 2322 , Loss: [0.9439992308616638, 0.515625]
Epoch: 2323 , Loss: [0.9205108880996704, 0.546875]
Epoch: 2324 , Loss: [0.9221033453941345, 0.53125]
Epoch: 2325 , Loss: [0.9410421848297119, 0.515625]
Epoch: 2326 , Loss: [0.9285097122192383, 0.515625]
Epoch: 2327 , Loss: [0.9952677488327026, 0.453125]
Epoch: 2328 , Loss: [0.8839461803436279, 0.5625]
Epoch: 2329 , Loss: [0.9230992794036865, 0.515625]
Epoch: 2330 , Loss: [0.8763535022735596, 0.578125]
Epoch: 2331 , Loss: [0.9264439344406128, 0.515625]
Epoch: 2332 , Loss: [0.958367109298706, 0.453125]
Epoch: 2333 , Loss: [0.9409526586532593, 0.453125]
Epoch: 2334 , Loss: [0.9579318165779114, 0.421875]
Epoch: 2335 , Loss: [0.8899752497673035, 0.578125]
Epoch: 2336 , Loss: [0.8560569286346436, 0.640625]
Epoch: 2337 , Loss: [0.9251296520233154, 0.453125]
Epoch: 2338 , Loss: [0.920655667781829

Epoch: 2483 , Loss: [0.916923999786377, 0.515625]
Epoch: 2484 , Loss: [0.9469661712646484, 0.375]
Epoch: 2485 , Loss: [0.9342843294143677, 0.421875]
Epoch: 2486 , Loss: [0.9056993722915649, 0.5]
Epoch: 2487 , Loss: [0.9024534821510315, 0.5]
Epoch: 2488 , Loss: [0.8726900815963745, 0.546875]
Epoch: 2489 , Loss: [0.9121231436729431, 0.515625]
Epoch: 2490 , Loss: [0.8974449634552002, 0.53125]
Epoch: 2491 , Loss: [0.9184365272521973, 0.46875]
Epoch: 2492 , Loss: [0.9296722412109375, 0.46875]
Epoch: 2493 , Loss: [0.9022246599197388, 0.46875]
Epoch: 2494 , Loss: [0.9212976694107056, 0.421875]
Epoch: 2495 , Loss: [0.921650230884552, 0.390625]
Epoch: 2496 , Loss: [0.9388603568077087, 0.484375]
Epoch: 2497 , Loss: [0.906395435333252, 0.484375]
Epoch: 2498 , Loss: [0.9186671376228333, 0.515625]
Epoch: 2499 , Loss: [0.8644784688949585, 0.546875]
Epoch: 2500 , Loss: [0.846688985824585, 0.5]
2 correctly classified among 64
Accuracy as of 2500 epochs: 3.125
Epoch: 2501 , Loss: [0.8770424127578735, 0

Epoch: 2645 , Loss: [0.9009413719177246, 0.515625]
Epoch: 2646 , Loss: [0.8467264771461487, 0.546875]
Epoch: 2647 , Loss: [0.8623999953269958, 0.5625]
Epoch: 2648 , Loss: [0.8633686304092407, 0.578125]
Epoch: 2649 , Loss: [0.8968241214752197, 0.484375]
Epoch: 2650 , Loss: [0.8655444383621216, 0.453125]
Epoch: 2651 , Loss: [0.8621988892555237, 0.578125]
Epoch: 2652 , Loss: [0.859153687953949, 0.5625]
Epoch: 2653 , Loss: [0.9142959117889404, 0.359375]
Epoch: 2654 , Loss: [0.8876513242721558, 0.46875]
Epoch: 2655 , Loss: [0.8565853238105774, 0.578125]
Epoch: 2656 , Loss: [0.8658933639526367, 0.515625]
Epoch: 2657 , Loss: [0.9067975282669067, 0.515625]
Epoch: 2658 , Loss: [0.8631432056427002, 0.53125]
Epoch: 2659 , Loss: [0.8450934290885925, 0.5625]
Epoch: 2660 , Loss: [0.8529312610626221, 0.5625]
Epoch: 2661 , Loss: [0.8761429786682129, 0.46875]
Epoch: 2662 , Loss: [0.8650774955749512, 0.53125]
Epoch: 2663 , Loss: [0.8889327049255371, 0.5]
Epoch: 2664 , Loss: [0.8622579574584961, 0.515625

KeyboardInterrupt: 